<a href="https://colab.research.google.com/github/mavallejos93/AnaliticaDatosPhyton/blob/master/spark_en_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 66 kB/s 
     |████████████████████████████████| 198 kB 27.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=6c82e49418e58f296ee19792f1e1d337cdd44d147471c600f95abbd02010924c
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [3]:
!wget https://raw.githubusercontent.com/camilousa/datasets/master/german_credit_data.csv

--2021-09-29 02:48:14--  https://raw.githubusercontent.com/camilousa/datasets/master/german_credit_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53393 (52K) [text/plain]
Saving to: ‘german_credit_data.csv’

german_credit_data. 100%[===================>]  52.14K  --.-KB/s    in 0.02s   

2021-09-29 02:48:14 (2.46 MB/s) - ‘german_credit_data.csv’ saved [53393/53393]



In [4]:
from pyspark.sql import SQLContext

In [5]:
sqlContext = SQLContext(spark)

In [6]:
df = sqlContext.read.csv("german_credit_data.csv", header=True, inferSchema=True)

In [7]:
df.show(40)

+---+---+------+---+-------+---------------+----------------+-------------+--------+-------------------+----+
|_c0|Age|   Sex|Job|Housing|Saving accounts|Checking account|Credit amount|Duration|            Purpose|Risk|
+---+---+------+---+-------+---------------+----------------+-------------+--------+-------------------+----+
|  0| 67|  male|  2|    own|             NA|          little|         1169|       6|           radio/TV|good|
|  1| 22|female|  2|    own|         little|        moderate|         5951|      48|           radio/TV| bad|
|  2| 49|  male|  1|    own|         little|              NA|         2096|      12|          education|good|
|  3| 45|  male|  2|   free|         little|          little|         7882|      42|furniture/equipment|good|
|  4| 53|  male|  2|   free|         little|          little|         4870|      24|                car| bad|
|  5| 35|  male|  1|   free|             NA|              NA|         9055|      36|          education|good|
|  6| 53| 

## Como convertir de spark a pandas

In [8]:
df3 = df.select("Age", "Sex", "Job", "Risk").where("Risk='good'")

In [9]:
import pandas as pd

In [10]:
df_pandas = df3.toPandas()

In [11]:
df_pandas

,Age,Sex,Job,Risk
0,67,male,2,good
1,49,male,1,good
2,45,male,2,good
3,35,male,1,good
4,53,male,2,good
...,...,...,...,...
695,50,male,2,good
696,31,female,1,good
697,40,male,3,good
698,38,male,2,good


In [12]:
df

DataFrame[_c0: int, Age: int, Sex: string, Job: int, Housing: string, Saving accounts: string, Checking account: string, Credit amount: int, Duration: int, Purpose: string, Risk: string]

Ejemplos de consultas en spark

In [13]:
df.where("Sex='male'").count()

690

In [14]:
df.where("Sex='female'").count()

310

In [15]:
df.groupby("Sex").count().show()

+------+-----+
|   Sex|count|
+------+-----+
|female|  310|
|  male|  690|
+------+-----+



In [16]:
df.groupby("Risk").count().show()

+----+-----+
|Risk|count|
+----+-----+
| bad|  300|
|good|  700|
+----+-----+



In [17]:
df.groupby("Sex", "Risk").count().show()

+------+----+-----+
|   Sex|Risk|count|
+------+----+-----+
|  male| bad|  191|
|  male|good|  499|
|female| bad|  109|
|female|good|  201|
+------+----+-----+



In [18]:
100 * 191/(191+499)

27.681159420289855

In [19]:
100 * 109/(109+201)

35.16129032258065

In [20]:
df.groupby("Sex").sum("Credit amount").show()

+------+------------------+
|   Sex|sum(Credit amount)|
+------+------------------+
|female|            892110|
|  male|           2379148|
+------+------------------+



In [21]:
df.groupby("Sex").avg("Credit amount").show()

+------+------------------+
|   Sex|avg(Credit amount)|
+------+------------------+
|female|2877.7741935483873|
|  male| 3448.040579710145|
+------+------------------+



In [22]:
df.groupby("Risk").sum("Credit amount").show()

+----+------------------+
|Risk|sum(Credit amount)|
+----+------------------+
| bad|           1181438|
|good|           2089820|
+----+------------------+



In [23]:
df.groupby("Sex", "Purpose").count().orderBy("Sex").show()

+------+-------------------+-----+
|   Sex|            Purpose|count|
+------+-------------------+-----+
|female|furniture/equipment|   74|
|female|            repairs|    5|
|female|          education|   24|
|female|           business|   19|
|female|    vacation/others|    3|
|female|domestic appliances|    6|
|female|           radio/TV|   85|
|female|                car|   94|
|  male|furniture/equipment|  107|
|  male|                car|  243|
|  male|           business|   78|
|  male|            repairs|   17|
|  male|    vacation/others|    9|
|  male|          education|   35|
|  male|           radio/TV|  195|
|  male|domestic appliances|    6|
+------+-------------------+-----+

